In [1]:
index_file = "/home/szymon/lingwy/nkjp/nkjp_index.txt"
nkjp_path = "/home/szymon/lingwy/nkjp/pełny/"
vecs_path = "/home/szymon/lingwy/nkjp/wektory/nkjp+wiki-lemmas-all-100-skipg-ns.txt/"
skladnica_sections_index = "/home/szymon/lingwy/nkjp/skladnica_znacz/sections.txt"
skladnica_path = "/home/szymon/lingwy/nkjp/skladnica_znacz/"
vecs_dim = 100
window_size = 4 # how many words to condider on both sides of the target
batch_size = window_size * 2
corp_runs = 2
learning_rate = 0.3
reg_rate = 0.005
points_per_neg_sample = 1

## Get Składnica & NKJP sentences

In [58]:
skl_ids = []
nkjp_orig_ids = []
nkjp_squeezed_ids = [] # Składnica uses "squeezed" version, w/o hyphens (-)

# This will be used later. NKJP id -> [ sent_ids ]
skl_nkjp_sent_ids = {}
# auxiliary, skl_id -> NKJP_id
nkjp_skl_ids = {}

with open(skladnica_sections_index) as index:
    for sec_name in index:
        skl_ids.append(sec_name.strip())
with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        nkjp_orig_ids.append(fragm_id)
        nkjp_squeezed_ids.append(fragm_id.replace('-', ''))
        
bonus_skladnica_mappings = {
                            'NKJP_1M_001000-DziennikPolski1980': 'DP1980',
                            'NKJP_1M_010200-ZycieWarszawyPLUSZycie': 'ZycieWarszawy_Zycie',
                            'NKJP_1M_012750-TrybunaPLUSTrybunaLudu': 'TrybunaLudu_Trybuna',
                            'NKJP_1M_1998': 'DP1998',
                            'NKJP_1M_1999': 'DP1999',
                            'NKJP_1M_2000': 'DP2000',
                            'NKJP_1M_2001': 'DP2001',
                            'NKJP_1M_2002': 'DP2002',
                            'NKJP_1M_2003': 'DP2003',
                            'NKJP_1M_%2E': '712-3-900001'
                           }

In [59]:
# This should print Składnica ids for which we don't know their NKJP equivalents.
# (hopefully, none)
for id in skl_ids:
    if id[len('NKJP_1M_'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_4scal-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_4scal-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_######-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_######-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id in bonus_skladnica_mappings:
        orig_nkjp_id = bonus_skladnica_mappings[id]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    else:
        print(id)

In [60]:
from lxml import etree
import os

In [61]:
# Get all the sents present in Składnica.
from operator import itemgetter

# NOTE this also contains empty lists where the sents were actually not annotated.
skl_sents = []

for skl_id in skl_ids:
    section_path = skladnica_path + skl_id
    # Each section has 1+ dirs of XML files.
    for dirname, _, __ in os.walk(section_path):
        if dirname != section_path: # ignore the dir itself
            # Get the XML files (each contains a sentence)
            for _, __, filenames in os.walk(dirname):
                for filename in filenames:
                    sent_id = filename[:-len('.xml')]
                    skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(sent_id)
                    
                    ###skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(tree.find(".//text"))
                    # Collect the list of sentence lemmas.
                    tree = etree.parse(dirname+'/'+filename)
                    sent = []
                    for elem in tree.iterfind('node[@chosen]'):
                        if elem.attrib['chosen'] == 'true' and elem.find("terminal") is not None:
                            sent.append((int(elem.attrib['from']),
                                            elem.find("terminal").find("base").text))
                    sent.sort(key=itemgetter(0))
                    sent = [token for num, token in sent]
                    skl_sents.append(sent)

In [62]:
print(skl_nkjp_sent_ids)

{'GazetaPomorska': ['morph_6.39-s', 'morph_6.76-s', 'morph_6.67-s', 'morph_6.22-s', 'morph_6.10-s'], 'DP1980': ['morph_17.81-s', 'morph_17.61-s', 'morph_17.75-s', 'morph_17.27-s', 'morph_17.90-s', 'morph_17.41-s', 'morph_3.38-s', 'morph_3.55-s', 'morph_3.47-s', 'morph_7.68-s', 'morph_7.28-s', 'morph_7.38-s', 'morph_7.55-s', 'morph_7.47-s', 'morph_7.17-s', 'morph_4.49-s', 'morph_4.65-s', 'morph_4.19-s', 'morph_8.56-s', 'morph_8.21-s', 'morph_8.63-s', 'morph_8.34-s'], '010-2-000000001': ['morph_3.20-s', 'morph_3.50-s', 'morph_3.36-s', 'morph_3.10-s', 'morph_3.3-s', 'morph_3.84-s', 'morph_3.80-s', 'morph_7.62-s', 'morph_7.28-s', 'morph_7.54-s', 'morph_6.63-s', 'morph_6.40-s'], 'ZycieWarszawy_Zycie': ['morph_146.82-s', 'morph_146.36-s', 'morph_146.13-s', 'morph_146.28-s', 'morph_146.75-s', 'morph_146.52-s', 'morph_146.17-s', 'morph_146.41-s', 'morph_65.47-s', 'morph_65.36-s', 'morph_65.66-s', 'morph_65.15-s', 'morph_44.66-s', 'morph_44.42-s', 'morph_44.47-s', 'morph_44.31-s', 'morph_44.7-s

In [63]:
print(skl_sents[:10])

[['bo', 'gdy', 'zostać', 'proboszcz', ',', 'nie', 'pozwolić', 'rada', 'parafialny', 'wprowadzić', 'się', 'procesja', 'do', 'plebania', 'i', 'kościół', '.'], ['-', 'taki', 'być', 'u', 'my', 'zwyczaj', 'od', 'pokolenie', '.'], [], ['po', 'awantura', 'z', 'rodzina', 'K', '.', 'wygrzebać', 'przeciwko', 'on', 'stary', 'sprawa', '.'], ['.'], ['osiągnąć', 'korzyść', 'prestiżowy', 'i', 'finansowy', '.'], [], ['twój', 'inicjatywa', ',', 'konkretny', ',', 'praktyczny', 'i', 'pożyteczny', ',', 'zyskać', 'uznanie', 'i', 'poparcie', '.'], [], []]


In [79]:
# Read the NKJP fragments, as lists of lemmas
unique_words = set()
words_count = 0
train_sents = []
rejected_sent_ids = []

with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        # tag is namespaced, .// for finding anywhere in the tree
        tokens_filepath = nkjp_path+fragm_id+'/ann_morphosyntax.xml'
        if not os.path.isfile(tokens_filepath):
            print('Note: cannot access {}'.format(fragm_id.strip()))
            continue
        tree = etree.parse(tokens_filepath)
        for sent_subtree in tree.iterfind('.//{http://www.tei-c.org/ns/1.0}s[@{http://www.w3.org/XML/1998/namespace}id]'):
            
            # Skip sentences analyzed in Składnica.
            sent_id = sent_subtree.get('{http://www.w3.org/XML/1998/namespace}id').replace(' ', '_')
            if fragm_id in skl_nkjp_sent_ids and sent_id in skl_nkjp_sent_ids[fragm_id]:
                rejected_sent_ids.append(fragm_id+':'+sent_id)
                continue
                
            sent_lemmas = []
            for elem in sent_subtree.iterfind('.//{http://www.tei-c.org/ns/1.0}f[@name]'):
                if elem.attrib['name'] == 'base':
                    sent_lemmas.append(elem[0].text) # first child <string>
            train_sents += [[]]
            for word in sent_lemmas:
                train_sents[-1].append(word)
                words_count += 1
                unique_words.add(word)
print('{} individual sents rejected, for {} known from test corpus'.format(len(rejected_sent_ids), len(skl_sents)))

19963 individual sents rejected, for 19963 known from test corpus


In [80]:
words_count

1542059

## Get word vectors

In [66]:
import numpy as np

In [67]:
first_line = True
word_n = 0
word_to_idx = {}
idx_to_word = {}

# we'll read those from the data file
vecs_count = 0
vecs_dim = 100

# Get the vector word labels (we'll get vectors themselves in a momment).
with open(vecs_path+"data") as vecs_file:
    for line in vecs_file:
        if first_line:
            # Read metadata.
            vecs_count = int(line.split(' ')[0])
            vecs_dim = int(line.split(' ')[1])
            first_line = False
            continue
        # Read lemma base forms.
        word_to_idx[line.split(' ')[0]] = word_n
        idx_to_word[word_n] = line.split(' ')[0]
        word_n += 1

In [68]:
word_vecs = np.loadtxt(vecs_path+"data", encoding="utf-8",
                       dtype=np.float32, # tensorflow's requirement
                       skiprows=1, usecols=tuple(range(1, vecs_dim+1)))

In [69]:
# Add the dummy boundary/unknown marker.
word_vecs = np.vstack([word_vecs, np.zeros((1,vecs_dim), dtype=np.float32)])
vecs_count += 1

In [70]:
# Get the word's vector, or the dummy marker.
def word_id(word):
    return word_to_idx[word] if word in word_to_idx else vecs_count-1

In [71]:
vecs_count, vecs_dim

(1549323, 100)

# Keras

In [72]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Embedding
from keras.optimizers import SGD

Using TensorFlow backend.
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [81]:
#
# Training corpus preparation.
#

from random import randint
from math import floor

# We need a special token for cases when the target word is near the start or end of sentence.
bound_token_id = vecs_count - 1 # the zero additional vector

sample_n = 0

train = np.zeros(((words_count + words_count // points_per_neg_sample) * corp_runs,
                  window_size * 2 + 1), dtype='int')
labels = np.ones(((words_count + words_count // points_per_neg_sample) * corp_runs,),
                dtype='int')

for run_n in range(corp_runs):
    sent_n = 0
    word_n = 0
        
    while sent_n < len(train_sents) and sample_n < train.shape[0]:
        # The positive sample.
        train[sample_n, window_size] = word_id(train_sents[sent_n][word_n]) # the target word
        for j in range(window_size): # its neighbors in the window_size
            train[sample_n, j] = (word_id(train_sents[sent_n][word_n-j-1]) if word_n-j-1 >= 0
                                  else bound_token_id)
            train[sample_n, window_size+j+1] = (word_id(train_sents[sent_n][word_n+j+1])
                                                if word_n+j+1 < len(train_sents[sent_n])
                                                else bound_token_id)
        # (the "one" label is the default)
        # labels[sample_n] = 1.0
        
        # The negative sample.
        sample_n += 1
        train[sample_n,] = np.random.permutation(train[sample_n-1,])
        # (replace two random words)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        labels[sample_n] = 0.0
                
        sample_n += 1
        word_n += 1
        # If words are exhausted, scan the sents for one that has some words.
        try:
            while word_n == len(train_sents[sent_n]):
                word_n = 0
                sent_n += 1
        # At the end of the corpus (== exhausted sents), break the loop.
        except IndexError:
            break

In [83]:
model = Sequential()                                                                                               
model.add(Embedding(vecs_count,
                    vecs_dim,
                    weights=[word_vecs],
                    input_length=window_size * 2 + 1,
                    trainable=False))                                                                              
model.add(LSTM(96))
model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = SGD(lr=learning_rate, decay=reg_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(train, labels)

Epoch 1/1
 687328/6168236 [==>...........................] - ETA: 24:53 - loss: 0.2680 - acc: 0.8909

In [26]:
model

In [27]:
X = np.asarray([word_id(w) for w in
                ['Niemcy', 'znienacka', 'wkroczyć', 'do', 'Francja', 'w', 'maj', 'kolejny', 'rok']],
               dtype='int')
X

array([  708, 21345,  5375,     8,   724,     0,   322,   164,    16])

In [36]:
Y = np.asarray([word_id(w) for w in
                ['kot', 'zimny', 'okrągły', 'start', 'do', 'w', 'Czechy', 'wykres', 'klub']],
               dtype='int')
Y
Y = np.asarray([word_id(w) for w in
                ['w', 'zimny', 'miesiąc', 'zakładać', 'żołnierz', 'do', 'szkoła', 'każdy', 'dzień']],
               dtype='int')
Y
Y = np.asarray([word_id(w) for w in
                ['w', 'kąt', 'izba', 'stać', 'urządzenie', 'kaflowy', 'z', 'wiele', 'garnek']],
               dtype='int')
Y

array([    0,  2605,   619,   123,  1300, 34251,     3,    92,  9152])

In [26]:
train.shape

(2646184, 9)

In [37]:
model.predict(np.atleast_2d(Y), batch_size=1)

array([[0.9920195]], dtype=float32)

# Torch

In [13]:
import torch
from torch.autograd import Variable

In [15]:
from random import randint
from math import floor

fragms_step = 1 # set to higher values if we want to skip some proportion of fragments
# We need a special token for cases when the target word is near the start or end of sentence.
bound_token_id = vecs_count - 1 # the zero additional vector

def skipgram_batches():
    for run_n in range(corp_runs):
        sent_n = 0
        word_n = 0
        
        target_n = 0 # relative to the current batch
        
        batch = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size), dtype=np.int32)
        
        while sent_n < len(fragms):
            for j in range(window_size):
                batch[target_n*window_size+j] = word_id(fragms[sent_n][word_n])
            # "Good" examples - words near the target (we will let TensorFlow randomize the "bad" ones)
            for j in range(window_size // 2):
                labels[target_n*window_size+j*2] = (word_id(fragms[sent_n][word_n-j-1]) if word_n-j-1 >= 0
                                                       else bound_token_id)
                labels[target_n*window_size+j*2+1] = (word_id(fragms[sent_n][word_n+j+1])
                                                         if word_n+j+1 < len(fragms[sent_n])
                                                         else bound_token_id)
                
            target_n += 1
            if target_n == (batch_size // window_size):
                yield batch, labels
                batch = np.ndarray(shape=(batch_size), dtype=np.int32)
                labels = np.ndarray(shape=(batch_size), dtype=np.int32)
                target_n = 0
                
            word_n += 1
            try:
                while word_n == len(fragms[sent_n]):
                    word_n = 0
                    sent_n += fragms_step
                    if (floor(sent_n / len(fragms) * 10)
                        > floor((sent_n-fragms_step) / len(fragms) * 10)):
                        print('{}0%'.format(floor(sent_n / len(fragms) * 10)), end=' ')
            except IndexError: # happens on the end of the corpus
                break
                
        batch[target_n:] = 0.0
        labels[target_n:, :] = 0.0
        yield batch, labels#, (run_n == corp_runs - 1)

In [36]:
classif_W1 = Variable(torch.randn(vecs_dim*2, vecs_dim*2), requires_grad=True)
classif_b1 = Variable(torch.randn(1, vecs_dim*2), requires_grad=True)
classif_W2 = Variable(torch.randn(vecs_dim*2, 1), requires_grad=True)
classif_b2 = Variable(torch.randn(1, 1), requires_grad=True)

In [38]:
import datetime

batch_n = 0

print('Training start:', datetime.datetime.now())
for batch, labels in skipgram_batches():
    # We get word indices, convert them to vectors.
    batch = word_vecs[batch] # main words
    pos_examples = word_vecs[labels]
    neg_examples = word_vecs[np.random.randint(vecs_count, size=pos_examples.shape)]
    
    pos_batch = Variable(torch.Tensor(np.hstack((batch, pos_examples))), requires_grad=False)
    pos_preds = ((pos_batch.mm(classif_W1) + classif_b1).sigmoid().mm(classif_W2)
                      + classif_b2).sigmoid()
    neg_batch = Variable(torch.Tensor(np.hstack((batch, pos_examples))), requires_grad=False)
    neg_preds = (((neg_batch.mm(classif_W1) + classif_b1).sigmoid()).mm(classif_W2)
                      + classif_b2).sigmoid()
    
    loss1 = ((- pos_preds).sum() / batch_size +
             # regularization:
                (classif_W1.abs().sum() + classif_b1.abs().sum() +
                 classif_W2.abs().sum() + classif_b2.abs().sum())
                * reg_rate)
    loss1.backward()
    loss2 = (neg_preds.sum() / (batch_size * 10) +
             # regularization:
                (classif_W1.abs().sum() + classif_b1.abs().sum() +
                 classif_W2.abs().sum() + classif_b2.abs().sum())
                * reg_rate)
    loss2.backward()
    if batch_n % 25000 == 0:
        print("Loss: {}".format(loss1 + loss2))
        #print(pos_preds)
        #print(neg_preds)
    
    classif_W1.data = learning_rate * classif_W1.grad.data
    classif_b1.data = learning_rate * classif_b1.grad.data
    classif_W2.data = learning_rate * classif_W2.grad.data
    classif_b2.data = learning_rate * classif_b2.grad.data
    classif_W1.grad.data.zero_()
    classif_b1.grad.data.zero_()
    classif_W2.grad.data.zero_()
    classif_b2.grad.data.zero_()
    
    batch_n += 1
print('Training end:', datetime.datetime.now())

Training start: 2018-02-21 22:48:45.389630
Loss: Variable containing:
 319.7979
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2369
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2369
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2370
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2371
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2371
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2368
[torch.FloatTensor of size 1]

10% Loss: Variable containing:
 1.2368
[torch.FloatTensor of size 1]

20% 30% 40% Loss: Variable containing:
 1.2369
[torch.FloatTensor of size 1]

50% Loss: Variable containing:
 1.2370
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2369
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2369
[torch.FloatTensor of size 1]

60% Loss: Variable containing:
 1.2370
[torch.FloatTensor of size 1]

Loss: Variable containing:
 1.2367
[torch.FloatTensor of size 1]

Loss: V

IndexError: too many indices for array

In [43]:
word = "być"
softmax = torch.nn.Softmax()
vec = Variable(torch.Tensor(
            np.hstack((np.broadcast_to(word_vecs[word_id(word), :], (vecs_count, vecs_dim)),
                       word_vecs))
         ), requires_grad=False)
pred = softmax((((vec.mm(classif_W1) + classif_b1).sigmoid()).mm(classif_W2)
                      + classif_b2))

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [41]:
pred.min()

Variable containing:
 0.3104
[torch.FloatTensor of size 1]

# Tensorflow (currently unused)

In [17]:
import tensorflow as tf
import math

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [113]:
tf.reset_default_graph()

with tf.device('/cpu:0'):
    # Model parameters: word embeddings and model weights & biases for each word.
    embeddings = tf.Variable(tf.random_uniform([vecs_count, vecs_dim], -1.0, 1.0))
    nce_weights = tf.Variable(tf.truncated_normal([vecs_count, vecs_dim],
                                                  stddev=1.0 / math.sqrt(vecs_dim)))
    nce_biases = tf.Variable(tf.zeros([vecs_count]))
    
    # The computation graph.
    inputs = tf.placeholder(tf.int32, shape=[batch_size])
    labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    embedding_layer = tf.nn.embedding_lookup(embeddings, inputs)
    # Note that word2vec has no "real" hidden layers apart from the embedding.
    
    # Number of random words to sample apart from the true target; the model should learn to
    # assign low probability to them given the context.
    negative_samples_n = batch_size
    
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                         biases=nce_biases,
                                         labels=labels,
                                         inputs=embedding_layer,
                                         num_sampled=negative_samples_n,
                                         num_classes=vecs_count))
    # Vanilla SGD seems to work here better - since we train practically a different word vector
    # each time, decaying momentum hinders training of later vectors before they can even be shown
    # to the net, especially in the case of Adagrad's vanishing updates.
    # (NOTE!) here we DO NOT touch the embeddings, we want to only learn nce_weights and biases!
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [114]:
import datetime

# we want to use those later:
trained_nce_weights = []
trained_nce_biases = []

with tf.Session() as sess:
    print('Training start:', datetime.datetime.now())
    tf.global_variables_initializer().run()
    i = 0
    for batch_inputs, batch_labels, is_last in skipgram_batches():
        if is_last:
            _, loss_val, trained_nce_weights, trained_nce_biases = sess.run(
                [optimizer, loss, nce_weights, nce_biases],
                feed_dict={inputs: batch_inputs, labels: batch_labels})
            print('Final loss:', loss_val)
            print('Training end:', datetime.datetime.now())
        else:
            _, loss_val = sess.run([optimizer, loss], feed_dict={inputs: batch_inputs,
                                                             labels: batch_labels})
            if (i % 250000 == 0):
                print('(loss: {})'.format(loss_val), end=' ')
        i += 1

Training start: 2018-02-07 19:02:51.118178
(loss: 675.9548950195312) 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% (loss: 2.3737263679504395) 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 10% 20% 30% 40% 50% (loss: 1.877175211906433) 60% 70% 80% 90% 100% Final loss: 2.2043426
Training end: 2018-02-07 19:25:10.739346


In [115]:
trained_nce_weights.shape, trained_nce_biases.shape

((55766, 100), (55766,))

In [116]:
pred_words = [rev_word_idx[i] if i < vecs_count-1 else ' ' for i in range(idxs.shape[1])][::-1] # reverse

In [117]:
word = "być"
vec = np.reshape(word_vecs[word_id(word), :], (1, vecs_dim))
prediction = np.dot(vec, np.transpose(trained_nce_weights))
prediction = np.add(prediction, trained_nce_biases)

In [118]:
word_id(word)

31166

In [119]:
idxs = np.argsort(prediction)

In [120]:
prediction.shape

(1, 55766)

In [122]:
pred_words[:100]

[' ',
 'kazirodczy',
 'teori',
 'nscrossen',
 'Bierówka',
 'hat-tricka',
 'bazgrać',
 'rozgościć',
 'niezmącony',
 'Valle',
 'łajać',
 '7-13',
 'Brett',
 'Silesii',
 'zaliczkowy',
 'Wojcieszów',
 'Sław',
 'odwiecznie',
 '²',
 'kwalifikator',
 'stulejka',
 'Eugen',
 'avi',
 'abażur',
 'wydeptywać',
 'Dederko',
 'płatowiec',
 'niedosłuch',
 'Kornasiewicz',
 'Krysiewicza',
 'Besbir',
 'separować',
 'Rumsfeld',
 'Kurowo',
 'obstrzał',
 'Galapagos',
 'odmownie',
 'Dłutów',
 'wolnomularski',
 'Liptak',
 'zbutwiały',
 'Gliwa',
 'Johana',
 'tia',
 'Cieślikowski',
 'doktrynerstwo',
 'nieokrzesany',
 '1440',
 'EG',
 'Lokia',
 'Elo',
 'budzetu',
 'UFK',
 '70-300',
 'wszystkowiedzący',
 'Burger',
 'ankiete',
 'Małastowskiej',
 'Kubiś',
 'rozpustny',
 'Galos',
 '75-300',
 'JKK',
 '637-12-23',
 'płaszczak',
 'wine',
 'ats',
 'żupan',
 'zauwazyl',
 'wykusz',
 "There's",
 'Solar',
 'skamleć',
 'szczotkować',
 'Karamazow',
 'gromnica',
 'dysgrafia',
 'łyżworolka',
 'Zarzycka',
 'Muskat',
 'choleryk',
 